# Model Building :

### Evaluation Metric

-  The Target is to get into top 5% in the kaggle Leaderboard thus, we need to get a Root Mean Squared Log Error or RMSLE < 0.1
<br />
<br />

<img src="_Images/RMSLE.png" width="400">
<br />

**Note that in the formulation X is the predicted value and Y is the actual value.**

## Importing Data and Dependencies

In [1]:
# Libraries

import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

# Data

train = pd.read_csv('Processed_DataSet/Final_train.csv')
train.head(3)

,MSSubClass,LotFrontage,LotArea,Street,Alley,LotShape,LandSlope,OverallQual,OverallCond,YearBuilt,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SalePrice
0,2.750250,5.831328,19.212182,0.730463,0.730463,1.540963,0.0,2.440268,1.820334,14.187527,...,0,0,1,0,0,0,0,1,0,12.247699
1,1.820334,6.221214,19.712205,0.730463,0.730463,1.540963,0.0,2.259674,2.440268,14.145138,...,0,0,1,0,0,0,0,1,0,12.109016
2,2.750250,5.914940,20.347241,0.730463,0.730463,0.000000,0.0,2.440268,1.820334,14.184404,...,0,0,1,0,0,0,0,1,0,12.317171


In [2]:
y_train = train.SalePrice
train.drop('SalePrice',axis=1,inplace=True)

## Defining a Validation Stratergy

In [3]:
#Define a cross Validation Function

n_folds = 5
def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

# Stratergy :
#### We will build, a Advanced Stacked Regressor for which we will first train various Base Models and then add a meta layer for Stacking

## Base Models

- Lasso Regression :

In [4]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

- Elastic Net Regression :

In [5]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

- Kernel Ridge Regression :

In [6]:
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

- Gradient Boosting Regression :

In [7]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

- XGBoost :

In [8]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, verbosity=0,
                             random_state =7, nthread = -1)

- LightGBM :

In [9]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

## Base Model Scoring

In [10]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 0.1116 (0.0074)



In [11]:
score = rmsle_cv(ENet)
print("\nElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


ElasticNet score: 0.1116 (0.0074)



In [12]:
score = rmsle_cv(KRR)
print("\nKernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Kernel Ridge score: 0.1152 (0.0075)



In [13]:
score = rmsle_cv(GBoost)
print("\nGradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Gradient Boosting score: 0.1166 (0.0083)



In [14]:
score = rmsle_cv(model_xgb)
print("\nXgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Xgboost score: 0.1161 (0.0072)



In [15]:
score = rmsle_cv(model_lgb)
print("\nLGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))


LGBM score: 0.1164 (0.0062)



## Model Stacking

### Average Based Model Class

In [16]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)  

### Averaging Class Score

In [17]:
averaged_score = AveragingModels(models = (ENet, GBoost, KRR, lasso))

score = rmsle_cv(averaged_score)
print("\nAveraged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Averaged base models score: 0.1087 (0.0076)



## Adding a Meta Class

In [18]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y.iloc[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

### Scoring of the Meta Class

In [19]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost, KRR),
                                                 meta_model = lasso)

score = rmsle_cv(stacked_averaged_models)
print("\nStacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))


Stacking Averaged models score: 0.1083 (0.0072)


### Final Ensemble Result :

In [20]:
#rmsle Func
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

#Stacked Regressor
stacked_averaged_models.fit(train.values, y_train)
stacked_pred = stacked_averaged_models.predict(train.values)

#XGBoost
model_xgb.fit(train, y_train)
xgb_pred = model_xgb.predict(train)

#LightGBM
model_lgb.fit(train, y_train)
lgb_pred = model_lgb.predict(train)

#Ensemble
print('Final Ensemble Score:')
print('{:.3f}'.format(rmsle(y_train,stacked_pred*0.70 +
               xgb_pred*0.15 + lgb_pred*0.15 )))

Final Ensemble Score:
0.076


## Conclusion :

#### The Final Score was 0.076 i.e < 0.1, and We can say that our Goal was Reached 